In [1]:
# Module containing improved interactive widgets for Earth Engine satellite processing

import ipywidgets as widgets
from IPython.display import display
import ee
import folium
from util import *
from loguru import logger

# # Authenticate and initialize Earth Engine
# ee.Authenticate()
# ee.Initialize()

In [24]:
class ML_Processor:
    
    def __init__(self):
        """Initialize the processor and create the initial widget"""
        # Check if Earth Engine is initialized
        try:
            ee.Initialize()
        except:
            print("Earth Engine needs to be initialized. Please run: ee.Authenticate() and ee.Initialize()")
        
        # Store processed data
        self.processed_data = None
        self.region = None
        self.collections = {}
        self.yearly_averages = {}
        self.mean_indices = {}
        
        # Create the input widget
        self._create_input_widget()
    
    def _create_input_widget(self):
        """Create the first widget for data selection and processing"""
        # Create dropdown widgets for variables
        years = list(range(1985, 2026))
        months = list(range(1, 13))
        satellites = ['Landsat', 'Sentinel']
        image_types = ["Cloudless Composite", "Mean Composite", "Least Cloudy Image"]
        
        # Year widgets
        self.year_widget = widgets.Dropdown(
            options=years,
            value=2020,
            description='Year:',
            disabled=False,
            layout=widgets.Layout(width='200px')
        )

        # Month widgets
        self.month_start_widget = widgets.Dropdown(
            options=months,
            value=1,
            description='Start Month:',
            disabled=False,
            layout=widgets.Layout(width='200px')
        )

        self.month_end_widget = widgets.Dropdown(
            options=months,
            value=5,
            description='End Month:',
            disabled=False,
            layout=widgets.Layout(width='200px')
        )

        # Satellite selection
        self.satellite_widget = widgets.Dropdown(
            options=satellites,
            value='Landsat',
            description='Satellite:',
            disabled=False,
            layout=widgets.Layout(width='200px')
        )

        # Image type
        self.image_type_widget = widgets.Dropdown(
            options=image_types,
            value="Cloudless Composite",
            description='Type of Image to Classify:',
            disabled=False,
            layout=widgets.Layout(width='200px')
        )
        # Region widget
        self.region_widget = widgets.Text(
            value="projects/ee-loucasdiamantboustead/assets/AOI_gaza",
            description='Region Asset:',
            disabled=False,
            layout=widgets.Layout(width='400px')
        )

        # Create process button
        self.process_button = widgets.Button(
            description='Process Data',
            button_style='primary',
            tooltip='Click to process satellite data',
            layout=widgets.Layout(width='150px'),
            icon='satellite'
        )
        self.process_button.on_click(self._on_process_click)

        # Create output area for processing feedback
        self.process_output = widgets.Output()
        self.map_output = widgets.Output()

        # Header and description
        header = widgets.HTML(value="<h2>Machine Learning Classification Processor</h2>")
        description = widgets.HTML(value="<p>Select parameters for satellite image processing and click 'Process Data'.</p>")

        # Layout organization
        inputs_box = widgets.VBox([
            self.year_widget, 
            self.month_start_widget, 
            self.month_end_widget,
            self.satellite_widget,
            self.image_type_widget,
            self.region_widget
        ])
        
        control_box = widgets.VBox([self.process_button])
        top_box = widgets.HBox([inputs_box, control_box])
        
        # Create the complete widget
        self.input_widget = widgets.VBox([header, description, top_box, self.process_output])
    

    # On Button CLick
    def _on_process_click(self, button):
        """Handle the process button click event"""
        # Clear previous output
        self.process_output.clear_output()
        
        # Get current parameter values
        year = self.year_widget.value
        start_month = self.month_start_widget.value
        end_month = self.month_end_widget.value
        satellite = self.satellite_widget.value
        image_type = self.image_type_widget.value
        region_asset = self.region_widget.value
        
        # Display parameters
        with self.process_output:
            print("-"*50)
            print(f"Parameters selected:")
            print(f"Year of classification: {year}")
            print(f"Month range: {start_month}-{end_month}")
            print(f"Satellite: {satellite}")
            print(f"Image type: {image_type}")
            print(f"Region asset: {region_asset}")
            print("-"*50) 

        self.region = ee.FeatureCollection(region_asset)
        self.image = self._generate_collection(year, start_month, end_month, satellite, self.region)     


        with self.process_output:
            self._display_folium_map()


    def _generate_collection(self, year, start_month=1, end_month=5, satellite='Landsat', region='', index_type='NDVI', image_type='Cloudless Composite'):
        """Process data for a range of years and return collections and averages."""
        # Initialize Landsat collections
        landsat5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
        landsat7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')
        landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
        sentinel2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')

        if satellite == 'Landsat':
            if year in [2003, 2012]:
                sat_collection = landsat7
                calculate_fn = lambda img: calculate_L5_index(img, index_type)
            elif year < 2013:
                sat_collection = landsat5
                calculate_fn = lambda img: calculate_L5_index(img, index_type)
            else:
                sat_collection = landsat8
                calculate_fn = lambda img: calculate_L8_index(img, index_type)
        elif satellite == 'Sentinel':
            sat_collection = sentinel2
            calculate_fn = lambda img: calculate_S2_index(img, index_type)
        
        # Process imagery
        collection = get_imagery(sat_collection, year, region, start_month, end_month)
        clipped = collection.map(lambda img: img.clip(region))
        index_added = clipped.map(calculate_fn)

        # Get the single image that will be used for classification
        if image_type == "Cloudless Composite":
            image = get_cloud_free_composite(index_added, satellite)
        elif image_type == "Mean Composite":
            image = get_mean_composite(index_added)
        elif image_type == "Least Cloudy Image":
            image = get_least_cloudy_image(index_added, satellite)


        return index_added.first()
        

    def _display_folium_map(self):
        """Display the processed image using Folium"""
        import folium
        
        # Get region bounds for map centering
        region_bounds = self.region.geometry().bounds().getInfo()
        region_coords = region_bounds['coordinates'][0]
        
        # Calculate center of the region
        lons = [coord[0] for coord in region_coords]
        lats = [coord[1] for coord in region_coords]
        center_lon = sum(lons) / len(lons)
        center_lat = sum(lats) / len(lats)

        # Create a folium map centered on the region
        my_map = folium.Map(location=[center_lat, center_lon], zoom_start=10)
        
        # Get satellite and image type for visualization parameters
        satellite = self.satellite_widget.value
        image_type = self.image_type_widget.value
        
        # Set up visualization parameters based on satellite type
        if satellite == 'Landsat':
            vis_params = {
                'bands': ['SR_B4', 'SR_B3', 'SR_B2'],  # True color RGB
                'min': 0,
                'max': 1,
                'gamma': 0
            }
            if image_type == "Cloudless Composite":
                vis_name = "Landsat Cloud-free Composite"
            elif image_type == "Mean Composite":
                vis_name = "Landsat Mean Composite"
            else:
                vis_name = "Landsat Least Cloudy Image"
        else:  # Sentinel
            vis_params = {
                'bands': ['B4', 'B3', 'B2'],  # True color RGB
                'min': 0,
                'max': 3000,
                'gamma': 1.4
            }
            if image_type == "Cloudless Composite":
                vis_name = "Sentinel Cloud-free Composite"
            elif image_type == "Mean Composite":
                vis_name = "Sentinel Mean Composite"
            else:
                vis_name = "Sentinel Least Cloudy Image"
        
        # Add the Earth Engine image to the map
        map_id_dict = self.image.getMapId(vis_params)
        folium.TileLayer(
            tiles=map_id_dict['tile_fetcher'].url_format,
            attr='Google Earth Engine',
            name=vis_name,
            overlay=True,
            control=True
        ).add_to(my_map)
        

        
        # Add layer control
        folium.LayerControl().add_to(my_map)
        
        # Display the map
        display(my_map)
        print("Map generated successfully!")







In [25]:
test = ML_Processor()
test.input_widget